# Amazon Nova text generation tasks

In this notebook we will demonstrate how to use [Amazon Nova *understanding* models](https://aws.amazon.com/ai/generative-ai/nova/) for various text generation tasks.

To execute the cells in this notebook you need to enable access to the following models on Bedrock:

* Amazon Nova Micro
* Amazon Nova Lite
* Anthropic Claude Haiku 3

see [Add or remove access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html) to manage the access to models in Amazon Bedrock.

Note: This notebook uses [Langchain](https://www.langchain.com/) to orchestrate the flow of the generative AI application.

## Setup

In [1]:
!pip install -U langchain-aws langchain-core langchain

In [2]:
import boto3
import langchain_core
import pydantic
import base64
import time

from langchain_aws import ChatBedrock

from langchain_core.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate

from botocore.exceptions import ClientError
from botocore.config import Config

In [3]:
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=Config(retries={'max_attempts': 20})
)

In [4]:
langchain_core.globals.set_debug(True) # Set to True for enabling debugging stack traces

## Quickly generate abstracts for your technical sessions with Nova Micro

[Amazon Nova Micro](https://docs.aws.amazon.com/ai/responsible-ai/nova-micro-lite-pro/overview.html) is the most lightweight, fast, and cost-effective text generation model of the Amazon Nova family. Its ideal for tasks such as:

* Q&A
* Summarization
* Translation
* Information Extraction

In this example we will use Amazon Nova Micro to quickly create high quality abstracts for technical presentations using as input the description of the topics that the presentation will cover

In [5]:
ABSTRACT_GENERATION_MODEL_PARAMETERS = {
    "max_tokens": 1500,
    "temperature": 0.7,
    "top_k": 20,
}

In [6]:
NOVA_MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0" # Cross Region Inference profile
NOVA_LITE_MODEL_ID = "us.amazon.nova-lite-v1:0" # Cross Region Inference profile
CLAUDE_MODEL_ID = "us.anthropic.claude-3-haiku-20240307-v1:0" # Cross Region Inference profile

bedrock_llm_nova_micro = ChatBedrock(
    model_id=NOVA_MICRO_MODEL_ID,
    model_kwargs=ABSTRACT_GENERATION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # Langchain object to interact with NOVA models through Bedrock

bedrock_llm_nova_lite = ChatBedrock(
    model_id=NOVA_LITE_MODEL_ID,
    model_kwargs=ABSTRACT_GENERATION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # We use Nova Lite for comparisson purposes

bedrock_llm_haiku3 = ChatBedrock(
    model_id=CLAUDE_MODEL_ID,
    model_kwargs=ABSTRACT_GENERATION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # We use Claude 3 for comparisson purposes

### Create a prompt for creating abstracts from the topics that will be included in the presentation

This prompt follows the guidelines published in: [https://aws.amazon.com/blogs/machine-learning/prompt-engineering-techniques-and-best-practices-learn-by-doing-with-anthropics-claude-3-on-amazon-bedrock/](https://aws.amazon.com/blogs/machine-learning/prompt-engineering-techniques-and-best-practices-learn-by-doing-with-anthropics-claude-3-on-amazon-bedrock/) and [https://docs.aws.amazon.com/nova/latest/userguide/prompting.html](https://docs.aws.amazon.com/nova/latest/userguide/prompting.html)

In [7]:
ABSTRACT_CREATION_SYSTEM_PROMPT_TEMPLATE = """
You are tech worker who is very skilled in writing abstracts for public speaking 
technical conferences. Your task is to write sessions abstract in a professional, 
concise way. 

You are very to the point when writing your abstracts and always use a semi-formal 
tone. 

Here are some rules for your consideration when writing the abstract: 

* Abstracts should clearly state the use case/problem and which public is being 
addressed or will benefit from attending the session, e.g. “In this session 
data scientist will learn how to build gen ai applications using AWS Bedrock”. 

* The abstract should clearly state the benefits/outcomes attendees will get out 
of the session; if a customer is presenting make sure to note what the customer 
will add to the conversation e.g. “Joins us to learn how WEG implemented, 
using SageMaker JumpStart pre-trained models, a text summarization system to 
reduce machinery troubleshooting from days to hours”. 

* If possible, close the abstract with a call to action e.g. “Join this session 
and learn how to build reliable ML systems at scale with AWS SageMaker”.

* Keep the abstract below 120 words (about 600 characters)

Here are some good examples for you: 

<examples> 

<good_abstract> 
Prompt engineering is the process of guiding large language models (LLMs) to 
produce desired outputs. In this session, get an overview of prompt engineering 
best practices and learn how to choose the most appropriate formats, phrases, 
words, and symbols to get the most out of generative AI solutions while improving 
accuracy and performance. This session uses the Claude 2 LLM as an example of how 
prompt engineering helps to solve complex customer use cases. Also learn how 
prompts can be integrated with your architecture and how to use API parameters for 
tuning the model parameters using Amazon Bedrock. 
</good_abstract> 

<good_abstract> 
Omics data holds great promise for improving health outcomes and accelerating 
scientific discovery, but analyzing large, complex genomic and multiomics datasets 
presents challenges. Storing, accessing, and deriving understanding from these data 
requires specialized tools. Learn how Amgen, a leading biopharma company, overcomes 
these hurdles using AWS HealthOmics, a purpose-built genomics cloud service, to 
transform different types of omics data into insights. Hear how Amgen uses the 
secure storage and scalable workflows of HealthOmics to efficiently generate 
insights from high-volume omics data. Using HealthOmics, researchers can transform 
genomics big data into useful knowledge, leading to advances in medicine and 
scientific breakthroughs. 
</good_abstract> 

<good_abstract> 
We are truly at an exciting inflection point in the widespread adoption of ML 
with the growth of generative AI applications. In this session, learn how to build 
your first generative AI application with key services such as Amazon Bedrock. 
Get hints and tips for getting started fast, and see example reference architectures 
for common use cases built with AWS AI and ML such as self-service customer support, 
text analysis, report generation, post-call analysis, and forecasting trends. 
</good_abstract> 

<good_abstract> 
Unexpected equipment failure is costly for industrial facilities. But scheduling 
maintenance too frequently wastes resources. In this session, hear from Koch Ag & 
Energy Solutions (KAES) on how they use Amazon Monitron to implement predictive 
maintenance across their industrial machinery. Learn how, with Amazon Monitron’s 
wireless sensors and machine learning, you can reduce unplanned downtime and 
transform maintenance operations to be data-driven and proactive. 
</good_abstract> 

</examples> 

Create a an abstract for a session that will present the content in <content> XML 
tags to a technical audience. The content is made up of several <source> sections. 
Be professional and concise and output the resulting abstract in the <abstract> XML 
tags. Do not add a premable to your answer.
"""


ABSTRACT_CREATION_USER_PROMPT_TEMPLATE = """
Create an abstract for a session that will presente the following content

<content> 
{session_content}
</content>

Please output your answer in Spanish
"""

For the content we compile some announcements from ["Whats new with AWS"](https://aws.amazon.com/new/?whats-new-content-all.sort-by=item.additionalFields.postDateTime&whats-new-content-all.sort-order=desc&awsf.whats-new-categories=*all). In this case we selected some announcements for Amazon Q.

You are encouraged to make your own compilation of topics

In [8]:
session_content = """
<source>
Starting today, Amazon Q Developer can also perform code reviews, automatically 
providing comments on your code in the IDE, flagging suspicious code patterns, 
providing patches where available, and even assessing deployment risk so you can 
get feedback on your code quickly.

Q Developer is a generative AI-powered assistant for designing, building, testing, 
deploying, and maintaining software. Its agents for software development have a 
deep understanding of your entire code repos, so they can accelerate many tasks 
beyond coding. By automating the first round of code reviews and improving review 
consistency, Q Developer empowers code authors to fix issues faster, streamlining 
the process for both authors and reviewers. With this new capability, Q Developer 
can help you get immediate feedback for your code reviews and code fixes where 
available, so you can increase the speed of iteration and improve the quality of 
your code.

This capability is available in the integrated development environment (IDE) through 
a new chat command: /review. You can start automating code reviews via the Visual 
Studio Code and IntelliJ IDEA Integrated Development Environments (IDEs) with both 
an Amazon Q Developer Free Tier or Pro Tier subscription. For more details on 
pricing, see Amazon Q Developer pricing. 
</source>

<source>
Today, Amazon Q Developer announces the general availability of a new agent that 
automates the process of generating unit tests. This agent can be easily initiated 
by using a simple prompt: “/test”. Once prompted, Amazon Q will use the knowledge 
of your project to automatically generate and add tests to your project, helping 
improve code quality, fast.

Amazon Q Developer will also ask you to provide consent before adding tests, 
allowing you to always stay in the loop so that no unintended changes are made. 
Automation saves the time and effort needed to write comprehensive unit tests, 
allowing you to focus on building innovative features. With the ability to quickly 
add unit tests and increase coverage across code, organizations can safely and more 
reliably ship code, accelerating feature development across the software development 
lifecycle.

Automatic unit test generation is generally available within the Visual Studio Code 
and JetBrains integrated development environments (IDEs) or in public preview as 
part of the new GitLab Duo with Amazon Q offering, in all AWS Regions where Amazon Q 
Developer is available. Learn more about unit test generation.
</source>

<source>
Starting today, Amazon Q Developer can document your code by automatically generating 
readme files and data-flow diagrams within your projects. 

Today, developers report they spend an average of just one hour per day coding. They 
spend most of their time on tedious, undifferentiated tasks such as learning 
codebases, writing and reviewing documentation, testing, managing deployments, 
troubleshooting issues or finding and fixing vulnerabilities. Q Developer is a 
generative AI-powered assistant for designing, building, testing, deploying, and 
maintaining software. Its agents for software development have a deep understanding 
of your entire code repos, so they can accelerate many tasks beyond coding. With this 
new capability, Q Developer can help you understand your existing code bases faster, 
or quickly document new features, so you can focus on shipping features for your 
customers.

This capability is available in the integrated development environment (IDE) through a 
new chat command: /doc . You can get started generating documentation within the 
Visual Studio Code and IntelliJ IDEA IDEs with an Amazon Q Developer Free Tier or 
Pro Tier subscription. For more details on pricing, see Amazon Q Developer pricing.
</source>

<source>
Starting today, you can build ML models using natural language with Amazon Q Developer, 
now available in Amazon SageMaker Canvas in preview. You can now get generative 
AI-powered assistance through the ML lifecycle, from data preparation to model 
deployment. With Amazon Q Developer, users of all skill levels can use natural language
 to access expert guidance to build high-quality ML models, accelerating innovation and 
 time to market.

Amazon Q Developer will break down your objective into specific ML tasks, define the 
appropriate ML problem type, and apply data preparation techniques to your data. 
Amazon Q Developer then guides you through the process of building, evaluating, and 
deploying custom ML models. ML models produced in SageMaker Canvas with Amazon Q 
Developer are production ready, can be registered in SageMaker Studio, and the code 
can be shared with data scientists for integration into downstream MLOps workflows.
</source>
"""

In [9]:
system_prompt_template = SystemMessagePromptTemplate.from_template(
    ABSTRACT_CREATION_SYSTEM_PROMPT_TEMPLATE,
    validate_template=True
)

user_prompt_template = HumanMessagePromptTemplate.from_template(
    ABSTRACT_CREATION_USER_PROMPT_TEMPLATE,
    input_variables=["session_content"],
    validate_template=True
)

abstract_generation_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    user_prompt_template
])

### Generate abstract using Amazon Nova

In [10]:
langchain_abstract_nova_micro = abstract_generation_prompt_template | bedrock_llm_nova_micro

start_time = time.time()
nova_completion_micro = langchain_abstract_nova_micro.invoke({ 
    "session_content":session_content,
})
time_micro = time.time() - start_time # Probably not the best way to compute execution time but it is convenient

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "session_content": "\n<source>\nStarting today, Amazon Q Developer can also perform code reviews, automatically \nproviding comments on your code in the IDE, flagging suspicious code patterns, \nproviding patches where available, and even assessing deployment risk so you can \nget feedback on your code quickly.\n\nQ Developer is a generative AI-powered assistant for designing, building, testing, \ndeploying, and maintaining software. Its agents for software development have a \ndeep understanding of your entire code repos, so they can accelerate many tasks \nbeyond coding. By automating the first round of code reviews and improving review \nconsistency, Q Developer empowers code authors to fix issues faster, streamlining \nthe process for both authors and reviewers. With this new capability, Q Developer \ncan help you get immediate feedback for your code reviews and code fixes where \navailable, so you can increa

In [11]:
langchain_abstract_nova_lite = abstract_generation_prompt_template | bedrock_llm_nova_lite

start_time = time.time()
nova_completion_lite = langchain_abstract_nova_lite.invoke({ 
    "session_content":session_content,
})
time_lite = time.time() - start_time # Probably not the best way to compute execution time but it is convenient

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "session_content": "\n<source>\nStarting today, Amazon Q Developer can also perform code reviews, automatically \nproviding comments on your code in the IDE, flagging suspicious code patterns, \nproviding patches where available, and even assessing deployment risk so you can \nget feedback on your code quickly.\n\nQ Developer is a generative AI-powered assistant for designing, building, testing, \ndeploying, and maintaining software. Its agents for software development have a \ndeep understanding of your entire code repos, so they can accelerate many tasks \nbeyond coding. By automating the first round of code reviews and improving review \nconsistency, Q Developer empowers code authors to fix issues faster, streamlining \nthe process for both authors and reviewers. With this new capability, Q Developer \ncan help you get immediate feedback for your code reviews and code fixes where \navailable, so you can increa

### Generate abstract using Anthropic Claude 3 Haiku

We now execute the same workload with Anthropic's Claude 3 Haiku model for comparisson purposes. Notice how easy it is to switch models using [Bedrock's Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). Also notice how we use the same prompt template as Nova's prompt template since the general principles for prompting apply to both models.

In [12]:
langchain_abstract_claude = abstract_generation_prompt_template | bedrock_llm_haiku3

start_time = time.time()
claude_completion = langchain_abstract_claude.invoke({ 
    "session_content":session_content,
})
time_claude = time.time() - start_time # Probably not the best way to compute execution time but it is convenient

### Side by side results

In [13]:
print(f"Amazon Nova Micro: \n\n{nova_completion_micro.content}\n\n\nAmazon Nova Lite: \n\n{nova_completion_lite.content}\n\n\n Anthropic Claude 3 Haiku: \n\n{claude_completion.content}")

Amazon Nova Micro: 

<abstract>
Descubre cómo Amazon Q Developer está revolucionando el desarrollo de software con nuevas capacidades de revisión de código, generación de pruebas de unidad y documentación automática. Con la capacidad de revisar automáticamente el código en tu entorno de desarrollo integrado (IDE), generar pruebas de unidad con solo una orden y documentar automáticamente tus proyectos, Q Developer acelera tareas más allá de la codificación. Únete a esta sesión para aprender cómo estas herramientas pueden aumentar tu velocidad de iteración y mejorar la calidad de tu código, todo desde tu IDE favorito. ¡Participa y acelera tu ciclo de desarrollo con Amazon Q Developer!
</abstract>


Amazon Nova Lite: 

<abstract>
Amazon Q Developer revoluciona el ciclo de desarrollo de software con capacidades 
de revisión de código automático, generación de pruebas unitarias, documentación 
automática y asistencia en la construcción de modelos de aprendizaje automático 
con lenguaje natu

In [14]:
print("\n\nAmazon Nova Micro\n")
print(f"Inference time: {nova_completion_micro.response_metadata['metrics']['latencyMs'][0]} miliseconds")
print(f"Execution time {time_micro} seconds")
print(f"Input tokens: {nova_completion_micro.usage_metadata['input_tokens']}")
print(f"Input tokens: {nova_completion_micro.usage_metadata['output_tokens']}")

print("\n\nAmazon Nova Lite\n")
print(f"Inference time: {nova_completion_lite.response_metadata['metrics']['latencyMs'][0]} miliseconds")
print(f"Execution time {time_lite} seconds")
print(f"Input tokens: {nova_completion_lite.usage_metadata['input_tokens']}")
print(f"Input tokens: {nova_completion_lite.usage_metadata['output_tokens']}")

print("\n\nAnthropic Claude Haiku 3\n")
print(f"Execution time {time_claude} seconds")
print(f"Input tokens: {claude_completion.usage_metadata['input_tokens']}")
print(f"Input tokens: {claude_completion.usage_metadata['output_tokens']}")



Amazon Nova Micro

Inference time: 1020 miliseconds
Execution time 1.1168313026428223 seconds
Input tokens: 1790
Input tokens: 127


Amazon Nova Lite

Inference time: 1472 miliseconds
Execution time 1.5764789581298828 seconds
Input tokens: 1790
Input tokens: 156


Anthropic Claude Haiku 3

Execution time 3.1755921840667725 seconds
Input tokens: 2023
Input tokens: 216
